# Getting started with flex flow in Azure

**Learning Objectives** - Upon completing this tutorial, you should be able to:

- Write an LLM application using a notebook and visualize the trace of your application.
- Convert the application into a flow and batch-run it against multiple lines of data.


## 0. Install dependent packages

In [ ]:
%%capture --no-stderr
%pip install -r ./requirements-azure.txt

## 1. Connection to workspace

### Configure credential

We are using `DefaultAzureCredential` to access the workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for other credentials if this does not work for you: [configure credential example](https://github.com/microsoft/promptflow/blob/main/examples/configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential does not work
    credential = InteractiveBrowserCredential()

### Connect to the workspace

We use a config file to connect to a workspace. The Azure ML workspace should be configured with a computer cluster. [Check this notebook for how to configure a workspace](https://github.com/microsoft/promptflow/blob/main/examples/configuration.ipynb)

In [ ]:
from promptflow.azure import PFClient

# Connect to the workspace
pf = PFClient.from_config(credential=credential)

### Create necessary connections
A connection helps securely store and manage secret keys or other sensitive credentials required for interacting with the LLM and other external tools, for example Azure Content Safety.

In this notebook, we will use the `basic` & `eval-code-quality` flex flow, which uses the connection `open_ai_connection`.  We need to set up the connection if we haven't added it before.

To prepare your Azure OpenAI resource, follow these [instructions](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal) and get your `api_key` if you don't have one.

Go to [workspace portal](https://ml.azure.com/), click `Prompt flow` -> `Connections` -> `Create`, then follow the instruction to create your own connections. 
Learn more on [connections](https://learn.microsoft.com/en-us/azure/machine-learning/prompt-flow/concept-connections?view=azureml-api-2).

## 2. Batch run the function as a flow with multi-line data.

Create a `flow.flex.yaml` file to define a flow whose entry points to the python function we defined.


In [ ]:
# Show the flow.flex.yaml content
with open("flow.flex.yaml") as fin:
    print(fin.read())

### Batch run with a data file (with multiple lines of test data)


In [ ]:
flow = "."  # Path to the flow directory
data = "./data.jsonl"  # Path to the data file

# Create a run with the flow and data
base_run = pf.run(
    flow=flow,
    data=data,
    column_mapping={
        "text": "${data.text}",
    },
    environment_variables={
        "AZURE_OPENAI_API_KEY": "${open_ai_connection.api_key}",
        "AZURE_OPENAI_ENDPOINT": "${open_ai_connection.api_base}",
    },
    stream=True,
)

In [ ]:
details = pf.get_details(base_run)
details.head(10)

## 3. Evaluate your flow
Then you can use an evaluation method to evaluate your flow. The evaluation methods are also flows which usually use an LLM to verify the produced output matches the expected output. 

### Setup model configuration with connection

When using Promptflow in Azure, create a model configuration object with connection name. 
The model config will connect to the cloud-hosted Promptflow instance while running the flow.

In [ ]:
from promptflow.core import AzureOpenAIModelConfiguration

model_config = AzureOpenAIModelConfiguration(
    connection="open_ai_connection",
    azure_deployment="gpt-4o",
)

### Evaluate the previous batch run
The **base_run** is the batch run we completed in step 2 above, for web-classification flow with "data.jsonl" as input.  The evaluation takes the outputs of that **base_run**, and uses an LLM to compare them to your desired outputs, and then visualizes the results.

In [ ]:
eval_flow = "../eval-code-quality/flow.flex.yaml"

eval_run = pf.run(
    flow=eval_flow,
    init={"model_config": model_config},
    data="./data.jsonl",  # path to the data file
    run=base_run,  # specify the base_run as the run you want to evaluate
    column_mapping={
        "code": "${run.outputs.output}",
    },
    stream=True,
)

In [ ]:
details = pf.get_details(eval_run)
details.head(10)

In [ ]:
import json

metrics = pf.get_metrics(eval_run)
print(json.dumps(metrics, indent=4))

In [ ]:
pf.visualize([base_run, eval_run])

## Next steps

You've successfully run your first flex flow and evaluated it. That's great!

You can check out more examples:
- [Basic Chat](https://github.com/microsoft/promptflow/tree/main/examples/flex-flows/chat-basic): demonstrates how to create a chatbot that can remember previous interactions and use the conversation history to generate the next message.